# Notebook: Análisis de Variables Predictoras vs 'canal_final'

Este notebook realiza el análisis variable a variable. La idea es:
1. Obtener los nombres de las columnas leyendo las primeras 4 filas del archivo.
2. Para cada variable predictora (excluyendo 'canal_final'):
   - Cargar la variable y la variable respuesta.
   - Para variables **numéricas**:
     - Realizar un binning (pd.qcut) y, a partir de la versión binned, calcular la tabla de contingencia para obtener las métricas Chi2, Chi2 p‑valor y Cramer’s V.
     - Calcular también métricas de ANOVA (F, p) y Eta Squared usando la variable original.
   - Para variables **categóricas**:
     - Obtener la lista de categorías y calcular la tabla de contingencia para obtener Chi2, p‑valor y Cramer’s V.
   - Entrenar un árbol de decisión sencillo (con `class_weight='balanced'`) para cada variable vs el target.  
     Para variables numéricas se usa la versión binned, y se extrae la métrica *weighted avg f1_score*.
3. Al final, se genera un DataFrame con una fila por variable y las métricas calculadas.

---

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# Función para calcular Eta Squared para variables numéricas
def calcular_eta_squared(x, y):
    overall_mean = np.mean(x)
    grupos = [x[y == cat] for cat in np.unique(y)]
    ss_between = sum(len(g) * (np.mean(g) - overall_mean) ** 2 for g in grupos)
    ss_total = sum((xi - overall_mean) ** 2 for xi in x)
    return ss_between / ss_total if ss_total != 0 else np.nan

# Función para evaluar una variable en relación con el target 'canal_final'
def evaluate_variable(x, y, var_tipo, n_bins=10):
    """
    Calcula métricas para una variable predictora frente a 'canal_final'.
    
    Parámetros:
      - x: Serie de la variable predictora.
      - y: Serie del target ('canal_final').
      - var_tipo: "numérica" o "categórica".
      - n_bins: Número de bins para variables numéricas.
    
    Retorna un diccionario con las métricas:
      - bins_grupos, ANOVA_F, ANOVA_p, Eta_Squared,
        Chi2, Chi2_p, Cramers_V, weighted_f1
    """
    metrics = {
        "bins_grupos": "no aplica",
        "ANOVA_F": "no aplica",
        "ANOVA_p": "no aplica",
        "Eta_Squared": "no aplica",
        "Chi2": "no aplica",
        "Chi2_p": "no aplica",
        "Cramers_V": "no aplica",
        "weighted_f1": "no aplica"
    }
    
    # Para la variable que se usará en el árbol de decisión:
    # Para numéricas se usará la versión binned; para categóricas, la original.
    if var_tipo == "numérica":
        # Binning: usamos pd.qcut
        try:
            x_binned, bins = pd.qcut(x, q=n_bins, duplicates='drop', retbins=True)
            metrics["bins_grupos"] = bins.tolist()
        except Exception as e:
            x_binned = None
            metrics["bins_grupos"] = "no aplica"
        
        # Métricas de ANOVA: usando la variable original
        grupos = []
        for cat in np.unique(y):
            group_data = x[y == cat]
            grupos.append(group_data.dropna())
        if len(grupos) > 1:
            try:
                f_val, p_val = stats.f_oneway(*grupos)
                metrics["ANOVA_F"] = f_val if not np.isnan(f_val) else "no aplica"
                metrics["ANOVA_p"] = p_val if not np.isnan(p_val) else "no aplica"
            except Exception as e:
                metrics["ANOVA_F"] = "no aplica"
                metrics["ANOVA_p"] = "no aplica"
        else:
            metrics["ANOVA_F"] = "no aplica"
            metrics["ANOVA_p"] = "no aplica"
        
        # Calcular Eta Squared
        try:
            eta2 = calcular_eta_squared(x, y)
            metrics["Eta_Squared"] = eta2 if not np.isnan(eta2) else "no aplica"
        except Exception as e:
            metrics["Eta_Squared"] = "no aplica"
        
        # Con la variable binned se calculan las métricas de Chi-cuadrado:
        if x_binned is not None:
            try:
                contingency = pd.crosstab(x_binned, y)
                if contingency.shape[0] > 1 and contingency.shape[1] > 1:
                    chi2, p, dof, expected = stats.chi2_contingency(contingency)
                    metrics["Chi2"] = chi2 if not np.isnan(chi2) else "no aplica"
                    metrics["Chi2_p"] = p if not np.isnan(p) else "no aplica"
                    n = contingency.sum().sum()
                    min_dim = min(contingency.shape) - 1
                    if n * min_dim != 0:
                        cramer_v = np.sqrt(chi2 / (n * min_dim))
                        metrics["Cramers_V"] = cramer_v
                    else:
                        metrics["Cramers_V"] = "no aplica"
                else:
                    metrics["Chi2"] = "no aplica"
                    metrics["Chi2_p"] = "no aplica"
                    metrics["Cramers_V"] = "no aplica"
            except Exception as e:
                metrics["Chi2"] = "no aplica"
                metrics["Chi2_p"] = "no aplica"
                metrics["Cramers_V"] = "no aplica"
        else:
            metrics["Chi2"] = "no aplica"
            metrics["Chi2_p"] = "no aplica"
            metrics["Cramers_V"] = "no aplica"
        
        # Para el árbol, usaremos la variable binned (si existe) convertida a string
        if x_binned is not None:
            X_feature = x_binned.astype(str)
        else:
            X_feature = x.astype(str)
    
    else:  # Variable categórica
        try:
            unique_vals = sorted(x.dropna().unique().tolist())
            metrics["bins_grupos"] = unique_vals
        except Exception as e:
            metrics["bins_grupos"] = "no aplica"
        
        # Las métricas ANOVA y Eta Squared no aplican para variables categóricas
        metrics["ANOVA_F"] = "no aplica"
        metrics["ANOVA_p"] = "no aplica"
        metrics["Eta_Squared"] = "no aplica"
        
        # Calcular Chi-cuadrado, p y Cramer’s V directamente
        try:
            contingency = pd.crosstab(x, y)
            if contingency.shape[0] > 1 and contingency.shape[1] > 1:
                chi2, p, dof, expected = stats.chi2_contingency(contingency)
                metrics["Chi2"] = chi2 if not np.isnan(chi2) else "no aplica"
                metrics["Chi2_p"] = p if not np.isnan(p) else "no aplica"
                n = contingency.sum().sum()
                min_dim = min(contingency.shape) - 1
                if n * min_dim != 0:
                    cramer_v = np.sqrt(chi2 / (n * min_dim))
                    metrics["Cramers_V"] = cramer_v
                else:
                    metrics["Cramers_V"] = "no aplica"
            else:
                metrics["Chi2"] = "no aplica"
                metrics["Chi2_p"] = "no aplica"
                metrics["Cramers_V"] = "no aplica"
        except Exception as e:
            metrics["Chi2"] = "no aplica"
            metrics["Chi2_p"] = "no aplica"
            metrics["Cramers_V"] = "no aplica"
        
        # Para el árbol, se utiliza la variable original convertida a string
        X_feature = x.astype(str)
    
    # Entrenar un árbol de decisión sencillo con class_weight='balanced'
    try:
        # Se deben codificar tanto la feature como el target
        le_feature = LabelEncoder()
        X_encoded = le_feature.fit_transform(X_feature)
        X_encoded = X_encoded.reshape(-1, 1)
        
        le_target = LabelEncoder()
        y_encoded = le_target.fit_transform(y)
        
        clf = DecisionTreeClassifier(class_weight='balanced', random_state=42)
        clf.fit(X_encoded, y_encoded)
        y_pred = clf.predict(X_encoded)
        f1 = f1_score(y_encoded, y_pred, average='weighted')
        metrics["weighted_f1"] = f1
    except Exception as e:
        metrics["weighted_f1"] = "no aplica"
    
    return metrics

# Función principal que:
# 1. Lee las primeras 4 filas del archivo para obtener los nombres de columnas.
# 2. Itera sobre cada variable predictora (excluyendo 'canal_final').
# 3. Para cada variable, carga dicha columna y 'canal_final' y calcula todas las métricas.
def main_analysis(file_path, target_column='canal_final', n_bins=10):
    # Leer las primeras 4 filas para obtener los nombres de las columnas
    df_head = pd.read_csv(file_path, nrows=4)
    all_columns = df_head.columns.tolist()
    
    # Se excluye la columna target para las variables predictoras
    predictor_columns = [col for col in all_columns if col != target_column]
    
    results = []
    
    # Iterar variable a variable
    for col in predictor_columns:
        try:
            # Cargar solo la columna predictora y la columna target
            df_var = pd.read_csv(file_path, usecols=[col, target_column])
        except Exception as e:
            print(f"Error al cargar la columna {col}: {e}")
            continue
        
        # Eliminar filas con datos faltantes en la variable o en el target
        df_var = df_var.dropna(subset=[col, target_column])
        
        # Determinar el tipo de variable
        if np.issubdtype(df_var[col].dtype, np.number):
            var_tipo = "numérica"
        else:
            var_tipo = "categórica"
        
        # Calcular todas las métricas para la variable actual
        metrics = evaluate_variable(df_var[col], df_var[target_column], var_tipo, n_bins=n_bins)
        
        result_row = {
            "variable": col,
            "tipo": var_tipo,
            "bins_grupos": metrics["bins_grupos"],
            "ANOVA_F": metrics["ANOVA_F"],
            "ANOVA_p": metrics["ANOVA_p"],
            "Eta_Squared": metrics["Eta_Squared"],
            "Chi2": metrics["Chi2"],
            "Chi2_p": metrics["Chi2_p"],
            "Cramers_V": metrics["Cramers_V"],
            "weighted_f1": metrics["weighted_f1"]
        }
        results.append(result_row)
    
    df_results = pd.DataFrame(results)
    return df_results

# Ejemplo de uso:
# file_path = 'ruta_al_archivo.csv'
# df_resultado_final = main_analysis(file_path, target_column='canal_final', n_bins=10)
# print(df_resultado_final)


Conclusiones

Binning y Métricas:
Para variables numéricas se realiza un binning y se calcula la tabla de contingencia para extraer Chi2, p‑valor y Cramer’s V. Además, se mantienen las métricas de ANOVA y Eta Squared utilizando la variable original.
Árbol de Decisión:
Se entrena un árbol de decisión sencillo (con class_weight='balanced') para cada variable predictora (usando la versión binned en el caso numérico) y se calcula el weighted avg f1_score, lo que brinda una medida adicional del poder discriminativo de la variable.
Procesamiento Iterativo:
Se realiza una lectura inicial de 4 filas para obtener la estructura del DataFrame y luego se procesa variable a variable, lo que permite trabajar con archivos muy grandes sin cargar todas las columnas de una vez.